# COGS 108 - Assignment 3: Data Privacy

## Important Reminders

- Do not change / update / delete any existing cells with 'assert' in them. These are the tests used to check your assignment. 
    - Changing these will be flagged for attempted cheating. 
- Do not rename this file.
- This assignment has hidden tests: tests that are not visible here, but that will be run on your submitted file. 
    - This means passing all the tests you can see in the notebook here does not guarantee you have the right answer!

## Overview

We have briefly discussed in lecture the importance and the mechanics of protecting individuals privacy when they are included in datasets. 

One method to do so is the Safe Harbor Method. The Safe Harbour method specifies how to protect individual's identities by telling us which tells us which information to remove from a dataset in order to avoid accidently disclosing personal information. 

In this assignment, we will explore web scraping, which can often include personally identifiable information, how identity can be decoded from badly anonymized datasets, and also explore using Safe Harbour to anonymize datasets properly. 

The topics covered in this assignment are mainly covered in the 'DataGathering' and 'DataPrivacy&Anonymization' Tutorial notebooks.

In [104]:
# Imports - these provided for you. Do not import any other packages
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup

## Part 1: Web Scraping 

### Scraping Rules

1) If you are using another organizations website for scraping, make sure to check the website's terms & conditions. 

2) Do not request data from the website too aggressively (quickly) with your program (also known as spamming), as this may break the website. Make sure your program behaves in a reasonable manner (i.e. acts like a human). One request for one webpage per second is good practice.

3) The layout of a website may change from time to time. Because of this, if you're scraping website, make sure to revisit the site and rewrite your code as needed.

### 1a) Web Scrape

We will first retrieve the contents on a page and examine them a bit.

Make a variable called `wiki`, that stores the following URL (as a string):
https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_population

Now, to open the URL, use `requests.get()` and provide `wiki` as its input. Store this in a variable called `page`.

After that, make a variable called `soup` to parse the HTML using `BeautifulSoup`. Consider that there will be a method from `BeautifulSoup` that you'll need to call on page to get the content from the page. 


In [105]:
wiki = 'https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_population'
page = requests.get(wiki)
soup = BeautifulSoup(page.text, "html.parser")
#raise NotImplementedError()

In [106]:
assert wiki
assert page
assert soup


### 1b) Checking Scrape Contents

Extract the title from the page and save it in a variable called `title_page`. 

Make sure you extract it as a string.

To do so, you have to use the soup object created in the above cell. 
Hint: from your soup variable, you can access this with `.title.string`.

Make sure you print out and check the contents of `title_page`.

Note that it should not have any tags (such as `<title>` included in it).

In [107]:
title_page = soup.title.string
print(title_page)
#raise NotImplementedError()

List of states and territories of the United States by population - Wikipedia


In [108]:
assert title_page
assert isinstance(title_page, str)


### 1c) Extracting Tables

In order to extract the data we want, we'll start with extracting a data table of interest.

Note that you can see this table by going to look at the link we scraped.

Use the `soup` object and call a method called `find`, which will and extract the first table in scraped webpage. Store this in the variable `right_table`. 

Note: you need to search for the name `table`, and set the `class_` argument as `wikitable sortable`.

In [109]:
right_table = soup.find("table", {"class":"wikitable sortable"})
print(right_table)


#raise NotImplementedError()

<table class="wikitable sortable" style="width:100%; text-align:center;">
<tbody><tr style="vertical-align: top;">
<th style="vertical-align: middle">Rank in the fifty states, 2018
</th>
<th style="vertical-align: middle">Rank in states &amp; territories, 2010
</th>
<th style="vertical-align: middle">Name
</th>
<th style="vertical-align: middle">Population estimate, July 1, 2018<br/><sup class="reference" id="cite_ref-5"><a href="#cite_note-5">[5]</a></sup>
</th>
<th style="vertical-align: middle">Census population, April 1, 2010<br/><sup class="reference" id="cite_ref-6"><a href="#cite_note-6">[6]</a></sup>
</th>
<th>Percent change, 2010–2018<br/><sup class="reference" id="cite_ref-7"><a href="#cite_note-7">[note 1]</a></sup>
</th>
<th>Absolute change, 2010-2018
</th>
<th style="vertical-align: middle">Total seats in the <a href="/wiki/United_States_House_of_Representatives" title="United States House of Representatives">U.S. House of Representatives</a>, 2013–2023
</th>
<th style="ve

In [110]:
assert right_table
assert isinstance(right_table, bs4.element.Tag)
assert right_table.name == 'table'

Now, Extract the data from the table into lists.

Note: This code provided for you. Do read through it and try to see how it works.

In [111]:
# CODE PROVIDED
# YOU SHOULD NOT HAVE TO EDIT
# BUT YOU WILL WANT TO UNDERSTAND
lst_a, lst_b, lst_c = [], [], []

for row in right_table.findAll('tr'):
    
    cells = row.findAll('td')
    
    # Skips rows that aren't 10 columns long (like the heading)
    if len(cells) != 12:
        continue

    # This catches when the name cells stops having a link
    #  and ends, skipping the last (summary rows)
    try:
        lst_a.append(cells[2].find('a').text)
        lst_b.append(cells[3].find(text=True))
        lst_c.append(cells[4].find(text=True))
    except:
        break

### 1d) Collecting into a dataframe

Create a dataframe `my_df` and add the data from the lists above to it. 
- `lst_a` is the state or territory name. Set the column name as `State`, and make this the index
- `lst_b` is the population estimate. Add it to the dataframe, and set the column name as `Population Estimate`
- `lst_c` is the census population. Add it to the dataframe, and set the column name as `Census Population`

In [112]:
my_df = pd.DataFrame(list(zip(lst_b, lst_c)), index=lst_a, columns=['Population Estimate', 'Census Population'])
my_df.index.name = 'State'
print(my_df)
#raise NotImplementedError()

                         Population Estimate Census Population
State                                                         
California                      39,557,045\n      37,254,523\n
Texas                           28,701,845\n      25,145,561\n
Florida                         21,299,325\n      18,801,310\n
New York                        19,542,209\n      19,378,102\n
Pennsylvania                    12,807,060\n      12,702,379\n
Illinois                        12,741,080\n      12,830,632\n
Ohio                            11,689,442\n      11,536,504\n
Georgia                         10,519,475\n       9,687,653\n
North Carolina                  10,383,620\n       9,535,483\n
Michigan                         9,995,915\n       9,883,640\n
New Jersey                       8,908,520\n       8,791,894\n
Virginia                         8,517,685\n       8,001,024\n
Washington                       7,535,591\n       6,724,540\n
Arizona                          7,171,646\n       6,39

In [113]:
assert isinstance (my_df, pd.DataFrame)
assert my_df.index.name == 'State'
assert list(my_df.columns) == ['Population Estimate', 'Census Population']


### 1e) Using the data
What is the Population Estimate of Texas? Save this answer to a variable called `texas_pop`
Notes:
- Extract this value programmatically from your dataframe (as in, don't set it explicitly, as `cf = 123`)
- You can use `.loc` to extract a particular value from a dataframe.
- The data in your dataframe will be strings - that's fine, leave them as strings (don't typecast).

In [114]:

texas_pop = my_df.iloc[1]['Population Estimate']
#raise NotImplementedError()

In [115]:
assert texas_pop


## Part 2: Identifying Data

Data Files:
- anon_user_dat.json
- employee_info.json

You will first be working with a file called 'anon_user_dat.json'. This file that contains information about some (fake) Tinder users. When creating an account, each Tinder user was asked to provide their first name, last name, work email (to verify the disclosed workplace), age, gender, phone # and zip code. Before releasing this data, a data scientist cleaned the data to protect the privacy of Tinder's users by removing the obvious personal identifiers: phone #, zip code, and IP address. However, the data scientist chose to keep each users' email addresses because when they visually skimmed a couple of the email addresses none of them seemed to have any of the user's actual names in them. This is where the data scientist made a huge mistake!

We will take advantage of having the work email addresses by finding the employee information of different companies and matching that employee information with the information we have, in order to identify the names of the secret Tinder users!

### 2a) Load in the 'cleaned' data 


Load the `anon_user_dat.json` json file into a pandas dataframe. Call it `df_personal`.

In [119]:
import json 

personal_json = json.load(open('anon_user_dat.json'))
df_personal = pd.DataFrame(personal_json)
df_personal
#df_personal = pd.read_json (r'anon_user_dat.json')
#print(df_personal)

#userJson = json.load(open('anon_user_dat.json'))
#df_personal = pd.DataFrame(userJson.get('data'))
#raise NotImplementedError()

,age,email,gender
0,60,gshoreson0@seattletimes.com,Male
1,47,eweaben1@salon.com,Female
2,27,akillerby2@gravatar.com,Male
3,46,gsainz3@zdnet.com,Male
4,72,bdanilewicz4@4shared.com,Male
...,...,...,...
995,3,pstroulgerrn@time.com,Female
996,49,kbasnettro@seattletimes.com,Female
997,75,pmortlockrp@liveinternet.ru,Male
998,81,sphetterq@toplist.cz,Male


In [120]:
assert isinstance(df_personal, pd.DataFrame)


### 2b) Check the first 10 emails 

Save the first 10 emails to a Series, and call it `sample_emails`. 
You should then print out this Series. 

The purpose of this is to get a sense of how these work emails are structured and how we could possibly extract where each anonymous user seems to work.


In [121]:
sample_emails = pd.Series(df_personal.iloc[0:11]['email'])
print(sample_emails)
#raise NotImplementedError()

0     gshoreson0@seattletimes.com
1              eweaben1@salon.com
2         akillerby2@gravatar.com
3               gsainz3@zdnet.com
4        bdanilewicz4@4shared.com
5       sdeerness5@wikispaces.com
6          jstillwell6@ustream.tv
7          mpriestland7@opera.com
8        nerickssen8@hatena.ne.jp
9              hparsell9@xing.com
10                acopasa@fda.gov
Name: email, dtype: object


In [122]:
assert isinstance(sample_emails, pd.Series)


### 2c) Extract the Company Name From the Email 

Create a function with the following specifications:
- Function Name: extract_company
- Purpose: to extract the company of the email (i.e., everything after the @ sign but before the .)
- Parameter(s): email (string)
- Returns: The extracted part of the email (string)
- Hint: This should take 1 line of code. Look into the find('') method. 

You can start with this outline:
```python 
def extract_company(email):
    return
```

Example Usage: 
- extract_company("larhe@uber.com") should return "uber"
- extract_company(“ds@cogs.edu”) should return “cogs”



In [123]:
def extract_company(email):
    return email[email.find('@')+1:email.find('.')]
extract_company('peter@gmail.com')
#raise NotImplementedError()

'gmail'

In [124]:
assert extract_company("gshoreson0@seattletimes.com") == "seattletimes"


With a little bit of basic sleuthing (aka googling) and web-scraping (aka selectively reading in html code) it turns out that you've been able to collect information about all the present employees/interns of the companies you are interested in. Specifically, on each company website, you have found the name, gender, and age of its employees. You have saved that info in employee_info.json and plan to see if, using this new information, you can match the Tinder accounts to actual names.

### 2d) Load in employee data 

Load the json file into a pandas dataframe. Call it `df_employee`.

In [125]:
#df_employee = pd.read_json (r'employee_info.json')
#print(df_employee)

employee_json = json.load(open('employee_info.json'))
df_employee = pd.DataFrame(employee_json)
df_employee

#raise NotImplementedError()

,company,first_name,last_name,gender,age
0,123-reg,Inglebert,Falconer,Male,42
1,163,Rafael,Bedenham,Male,14
2,163,Lemuel,Lind,Male,31
3,163,Penny,Pennone,Female,45
4,163,Elva,Crighton,Female,52
...,...,...,...,...,...
995,zdnet,Guido,Comfort,Male,46
996,zdnet,Biron,Malkinson,Male,48
997,zimbio,Becka,Waryk,Female,27
998,zimbio,Andreana,Ladewig,Female,34


In [126]:
assert isinstance(df_employee, pd.DataFrame)


### 2e) Match the employee name with company, age, gender 

Create a function with the following specifications:
- Function name: employee_matcher
- Purpose: to match the employee name with the provided company, age, and gender
- Parameter(s): company (string), age (int), gender (string)
- Returns: The employee first_name and last_name like this: return first_name, last_name 
- Note: If there are multiple employees that fit the same description, first_name and last_name should return a list of all possible first names and last name i.e., ['Desmund', 'Kelby'], ['Shepley', 'Tichner']. Note that the names of the individuals that would produce this output are 'Desmund Shepley' and 'Kelby Tichner'.

Hint:
There are many different ways to code this. An unelegant solution is to loop through `df_employee` 
   and for each data item see if the company, age, and gender match
   i.e., 
   ```python
   for i in range(0, len(df_employee)):
             if (company == df_employee.ix[i,'company']):
   ```
   
However! The solution above is very inefficient and long, so you should try to look into this:
Google the df.loc method: It extracts pieces of the dataframe
   if it fulfills a certain condition.
   i.e., 
   
```python
df_employee.loc[df_employee['company'] == company]
```

If you need to convert your pandas data series into a list, you can do ```list(result)``` where result is a pandas "series"

You can start with this outline:
```python
def employee_matcher(company, age, gender):
    return first_name, last_name
```


In [127]:
def employee_matcher(company, age, gender):
    
    matcher = df_employee.loc[(df_employee['company'] == company)& (df_employee['age'] == age) & (df_employee['gender'] == gender)]
    
    #first_names.append(matcher.first_name.tolist())
    #last_names.append(matcher.last_name.etolist())
    
    first_name = list(matcher['first_name'])
    last_name = list(matcher['last_name'])    
    
    return first_name, last_name

#raise NotImplementedError()

In [128]:
assert employee_matcher("google", 41, "Male") == (['Maxwell'], ['Jorio'])
assert employee_matcher("salon", 47, "Female") == (['Elenore'], ['Gravett'])


### 2f) Extract all the private data 

- Create 2 empty lists called `first_names` and `last_names`
- Loop through all the people we are trying to identify in df_personal
- Call the `extract_company function` (i.e., `extract_company(df_personal.ix[i, 'email'])` )
- Call the `employee_matcher` function 
- Append the results of `employee_matcher` to the appropriate lists (`first_names` and `last_names`)


In [129]:
first_names = []
last_names = []
for i in range(len(df_personal)):
    email = df_personal.loc[i, 'email']
    age = df_personal.loc[i, 'age']
    gender = df_personal.loc[i, 'gender']
    company = extract_company(email)
    firstname, lastname = employee_matcher(company, age, gender)
    first_names.append(firstname)
    last_names.append(lastname)


In [130]:
assert first_names[45:50]== [['Justino'], ['Tadio'], ['Kennith'], ['Cedric'], ['Amargo']]
assert last_names[45:50] == [['Corro'], ['Blackford'], ['Milton'], ['Yggo'], ['Grigor']]


### 2g) Add the names to the original 'secure' dataset! 

We have done this last step for you below, all you need to do is run this cell.

For your own personal enjoyment, you should also print out the new `df_personal` with the identified people. 

In [131]:
df_personal['first_name'] = first_names
df_personal['last_name'] = last_names

We have now just discovered the 'anonymous' identities of all the registered Tinder users...awkward.

## Part 3: Anonymize Data

You are hopefully now convinced that with some seemingly harmless data a hacker can pretty easily discover the identities of certain users. Thus, we will now clean the original Tinder data ourselves according to the Safe Harbor Method in order to make sure that it has been *properly* cleaned...

### 3a) Load in personal data 

Load the `user_dat.csv` file into a pandas dataframe. Call it `df_users`.

In [132]:
df_users = pd.read_csv('user_dat.csv')
df_users
#raise NotImplementedError()

,age,email,first_name,gender,last_name,ip_address,phone,zip
0,34,clilleymanlm@irs.gov,Carly,Female,Duckels,229.46.197.198,(445)515-0719,70397
1,87,parnecke9a@furl.net,Prisca,NaN,Le Friec,60.255.20.98,(962)747-5149,71965
2,60,ldankersley7j@mysql.com,Lauree,Female,Meineking,65.148.56.18,(221)690-1264,47946
3,47,kcattrollma@msn.com,Karoly,NaN,Hoyles,207.40.101.214,(203)282-1167,29063
4,85,rchestney60@dailymotion.com,Rona,Female,St. Quentin,177.12.128.156,(703)482-9159,68872
...,...,...,...,...,...,...,...,...
945,57,aleadbitterq8@oracle.com,Antons,Male,Guild,193.63.94.212,(577)985-9746,22812
946,23,cbrackennn@weather.com,Claudianus,Male,Welch,191.230.52.187,(885)626-9235,31522
947,33,jcoldbathe20@spotify.com,Jonell,Female,Salzberger,97.85.91.97,(496)413-2916,34219
948,47,lbonallack5f@sun.com,Locke,Male,Leeburne,233.219.2.228,(898)149-7657,75153


In [133]:
assert isinstance(df_users, pd.DataFrame)


### 3b) Drop personal attributes 

Remove any personal information, following the Safe Harbour method.
Based on the Safe Harbour method, remove any columns from `df_users` that contain personal information. 

Note that details on the Safe Harbour method are covered in the Tutorials.



In [134]:
df_users.drop(columns = ['email', 'first_name', 'last_name', 'ip_address', 'phone'], inplace = True)

In [135]:
assert len(df_users.columns) == 3


### 3c) Drop ages that are above 90 

Safe Harbour rule C: Drop all the rows which have age greater than 90 from `df_users`.

In [136]:
df_users = df_users[df_users['age'] <= 90]

In [137]:
assert df_users.shape == (943, 3)


### 3d) Load in zip code data 

Load the zip_pop.csv file into a (different) pandas dataframe. Call it `df_zip`.

Note that the zip data should be read in as strings, not ints, as would be the default. 

In read_csv, use the parameter `dtype` to specify to read `zip` as str, and `population` as int.

In [138]:
df_zip = pd.read_csv('zip_pop.csv', dtype = {'zip':str, 'population':int})
df_zip
#raise NotImplementedError()

,zip,population
0,01001,16769
1,01002,29049
2,01003,10372
3,01005,5079
4,01007,14649
...,...,...
33087,99923,87
33088,99925,819
33089,99926,1460
33090,99927,94


In [139]:
assert isinstance(df_zip, pd.DataFrame)


### 3e) Sort zipcodes into "Geographic Subdivision" 

The Safe Harbour Method applies to "Geographic Subdivisions"as opposed to each zipcode itself. 

Geographic Subdivision: All areas which share the first 3 digits of a zip code

Count the total population for each geographic subdivision

Warning: you have to be savy with a dictionary here

To understand how a dictionary works, check the section materials, use google and go to discussion sections!

Instructions: 
- Create an empty dictionary: ```zip_dict = {}```
- Loop through all the zip_codes in df_zip
- Create a dictionary key for the first 3 digits of a zip_code in zip_dict
- Continually add population counts to the key that contains the 
    same first 3 digits of the zip code

To extract the population you will find this code useful:

```python
population = list(df_zip.loc[df_zip['zip'] == zip_code]['population'])
```

To extract the first 3 digits of a zip_code you will find this code useful:
```python
int(str(zip_code)[:3])
```
**Note**: this code may take some time (many seconds, up to a minute or two) to run



In [187]:
zip_dict = {}
zipcodes = list(df_zip['zip'])
populations = list(df_zip.loc[df_zip['zip'] == zipcodes]['population'])

for i in range(len(zipcodes)):
    zipcode = int(zipcodes[i][:3])
    population = populations[i]
    
    if zipcode in zip_dict:
        zip_dict[zipcode] += population
    else:
        zip_dict[zipcode] = population


In [188]:
assert isinstance(zip_dict, dict)
assert zip_dict[100] == 1502501


### 3f) Explain this code excerpt 

```python
# In the cell below, explain in words what what the following line of code is doing:
population = list(df_zip.loc[df_zip['zip'] == zip_code]['population'])
```

Note: you do not have to *use* this line of code at this point in the assignment.

It is one of the lines provided to you in 3e. Here, just write a quick comment on what it does. This question will not be graded, but it's important to be able to read other people's code.

Sets the variable population to a casted list from the zip dataframe of elements in the zip column which is also updated to equal zip_code from the population column

### 3g) Masking the Zip Codes 

In this part, you should write a for loop, updating the df_users dataframe.

Go through each user, and update their zip-code, to Safe Harbour specifications:

- If the user is from a zip code for the which the "Geographic Subdivision" is less than equal to 20000, change the zip code to 0 
- Otherwise, change the zip code to be only the first 3 numbers of the full zip cide
- Do all this re-writting the zip_code columns of the `df_users` DataFrame

Hints: This will be several lines of code, looping through the DataFrame, getting each zip code, checking the geographic subdivision with the population in `zip_dict`, and setting the `zip_code` accordingly. 

In [189]:
for index, row in df_users.iterrows():
    zip_code = row['zip']
    re_zip = int(str(zip_code)[:3])
    if (zip_dict.get(re_zip)) <= 20000:
        df_users.replace(to_replace = zip_code, value = 0, inplace = True)
    else:
        df_users.replace(to_replace = zip_code, value = re_zip, inplace = True)
        

In [190]:
assert len(df_users) == 943
assert sum(df_users.zip == 0) == 5 or sum(df_users.zip == 0) == 6
assert df_users.loc[671, 'zip'] == 285


AssertionError: 

### 3h) Save out the properly anonymized data to json file 

Save out df_users as a json file, called `real_anon_user_dat.json`

In [186]:
df_users.to_json('real_anon_user_dat.json')

In [149]:
assert isinstance(pd.read_json('real_anon_user_dat.json'), pd.DataFrame)

Congrats, you're done! The users identities are much more protected now.

## Re-start & run all cells to be sure that everything passes, validate, and submit on DataHub!